In [1]:
import numpy as np
eps = 1e-3

In [2]:
def gradiente(f,xk):
    # Código que calcula el gradiente de la 
    # función f en el punto xk.
    size = len(xk)
    grad = np.zeros(size)
    for i in range(size):
        # Derivada mediante diferencia central
        x_f = xk.copy(); x_f[i] += eps
        x_b = xk.copy(); x_b[i] -= eps
        grad[i] = (f(x_f) - f(x_b)) / (2 * eps)
    return grad

In [3]:
def hessiana(f,xk):
    # Código que calcula la hessiana
    size = len(xk)
    hess = np.zeros((size, size))
    for j in range(size):
        # Segunda derivada
        dx_ff = xk.copy(); dx_ff[j] += eps;
        dx_bb = xk.copy(); dx_ff[j] += eps;
        grad_f = gradiente(f, dx_ff)
        grad_b = gradiente(f, dx_bb)
        for i in range(size):
            # Primera derivada
            hess[i, j] = (grad_f[i] - grad_b[i]) / (2 * eps)
    return hess

In [4]:
def condiciones_optimalidad(f,xk):
    # Código que regresa si el punto xk cumple 
    # con las condiciones de optimalidad
    grad = gradiente(f, xk)
    hess = hessiana(f, xk)
    # calculamos si el gradiente es 0
    if np.allclose(grad, 0, rtol = eps, atol = eps) :
        #Calculamos si es semipositiva definida
        """
        Solo funciona si se busca pos-def
        try:
            np.linalg.cholesky(hess)
        except: pass
        """
        eig_val, eig_vec = np.linalg.eig(hess)
        if(np.all(eig_val) >= 0):
            return True
    return False

In [5]:
def mk(f,xk):
    # Código que genera la función de aproximación
    # mk para el algoritmo de Región de confianza.
    grad = gradiente(f, xk)
    hess = hessiana(f, xk)
    # p = - np.linalg.inv(hess) @ grad # Solo funciona si hess es pos-def
    if (np.all(grad) == 0):
        p = np.zeros(len(grad))
    else:
        p = - eps * grad / np.linalg.norm(grad)
    return f(xk) + p.T @ grad + 0.5 * (p.T @ hess) @ p

In [6]:
#Función a probar
def f(x):
    #(x1 + x2 + ... + xn)^2
    return pow(np.sum(x),2)

# Gradiente de la función
def df(x):
    # 2 * (x1 + x2 + ... + xn)
    df = 2 * np.sum(x) * np.ones(len(x))
    return df

# Hessiana de la función
def dff(x):
    # [[2, ..., 2],...,[2, ..., 2]]
    size = len(x)
    dff = 2 * np.ones((size, size))
    return dff

In [7]:
# Prueba
# xk = np.array([-2.0, 2.0, -1.0, 1.0])
# xk = np.array([0.0, 0.0, 0.0, 0.0])
xk = np.array([2.0, 1.0, 4.0, 3.0])
print("f(x):",f(xk))

# Calculamos la gradiente y comprobamos
grad = gradiente(f, xk)
grad_true = df(xk)
print("Gradiente:", grad)
#print(np.isclose(grad, grad_true, rtol = eps, atol = eps))

# Calculamos la hessiana y comprobamos
hess = hessiana(f, xk)
hess_true = dff(xk)
print("Hessiana:", hess, sep='\n')
# print(np.isclose(hess, hess_true, rtol = eps**(1/2), atol = eps**(1/2)))

# Vemos si cumple con las condiciones de optimalidad
print("Optimalidad:", condiciones_optimalidad(f,xk))

# Calculamos mk
print("Mk:",mk(f, xk))

f(x): 100.0
Gradiente: [20. 20. 20. 20.]
Hessiana:
[[2.00000001 2.00000001 2.         2.        ]
 [2.00000001 2.00000001 2.         2.        ]
 [2.         2.         1.99999999 1.99999999]
 [2.00000001 2.00000001 2.         2.00000001]]
Optimalidad: False
Mk: 99.96000400000001
